# Variable Lens Sequence

A system with multiple lenses in sequence. The number of lenses of each type is an input parameter of the script:

* Number of plano convex lenses
* Number of biconvex lenses
* Number of reverse plano

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

import math
import numpy as np

### DESIGN ###

# Design parameters
square_size = 30
lens_diameter = math.sqrt(2)*square_size
focal_length = 45.

# Number of each lens type
nplano = 1
nbiconvex = 2
nrplano = 1

# Mechanical sizes
lens_min_thickness = 1.2
lens_spacing = 3.

### MODEL ###

# Parametric surfaces 
surface_plano = tlm.Parabola(lens_diameter, tlm.parameter(-0.002))
surface_biconvex = tlm.Parabola(lens_diameter, tlm.parameter(0.002))

# Lenses
lens_plano = tlm.PlanoLens(
    surface_plano,
    n = (1.0, 1.5),
    outer_thickness = lens_min_thickness,
    reverse=False,
)

lens_biconvex = tlm.BiLens(
    surface_biconvex,
    n = (1.0, 1.5),
    outer_thickness = lens_min_thickness,
)
    
lens_rplano = tlm.PlanoLens(
    surface_plano,
    n = (1.0, 1.5),
    outer_thickness = lens_min_thickness,
    reverse=True,
)

optics = nn.Sequential(
    tlm.PointSourceAtInfinity(beam_diameter=0.9*lens_diameter),
    tlm.Gap(10.),
    *[
        tlm.Gap(lens_spacing),
        lens_plano,
    ]*nplano,
    *[
        tlm.Gap(lens_spacing),
        lens_biconvex,
    ] *nbiconvex,
    *[
        tlm.Gap(lens_spacing),
        lens_rplano,
    ]*nrplano,
    tlm.Gap(focal_length),
    tlm.FocalPoint(),
)

print("Lens design")
print("Square size", square_size)
print("Lens diameter", lens_diameter)
print("Configuration", nplano, nbiconvex, nrplano)
print("lens_min_thickness", lens_min_thickness)
print("lens_spacing", lens_spacing)

tlm.show(optics, mode="2D")
tlm.show(optics, mode="3D")

In [ ]:
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=3e-4),
    sampling = {"dim": 2, "dtype": torch.float64, "base": 10},
    num_iter = 100
).plot()

def print_thickness(lens_name, lens):
    # TODO thickness at a specific radial distance
    print(f"{lens_name: <25} inner: {lens.inner_thickness().item():.3f} outer: {lens.outer_thickness().item():.3f}")

print_thickness("Plano-convex", lens_plano)
print_thickness("Bi-convex", lens_biconvex)
print_thickness("Reverse plano-convex", lens_rplano)

In [ ]:
# TODO

# can I regu by referencing params directly?
def regu_equalparam(optics):
    params = torch.cat([param.view(-1) for param in optics.parameters()])
    return torch.pow(torch.diff(1000*torch.abs(params)).sum(), 2)
    
def regu_equalthickness(optics):
    t0 = optics.plano.inner_thickness()
    t1 = optics.biconvex.inner_thickness()
    return 100*torch.pow(t0 - t1, 2)



In [ ]:
from IPython.display import display

display(tlm.export.lens_to_part(lens_plano))
display(tlm.export.lens_to_part(lens_biconvex))